In [50]:
test = """
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
"""

from functools import partial

def parse_part(part):
    part_scores = {}
    for part_score in part.strip('{}').split(','):
        category, score = part_score.split('=')
        part_scores[category] = int(score)

    return part_scores

def process_rule(part, key, comparison, amount, result):
    value = part[key]
    if (comparison == '<' and value < amount) or (comparison == '>' and value > amount):
        return result
    

def parse_workflow(workflow):
    name, rules = workflow.split('{')
    rules = rules.strip('}').split(',')
    rule_funcs = []
    for rule in rules:
        if ':' not in rule:
            rule_funcs.append(lambda part : rule)
            continue

        condition, result = rule.split(':')
        key = condition[0]
        comparison = condition[1]
        amount = int(condition[2:])
        rule_func = partial(process_rule, key=key, comparison=comparison, amount=amount, result=result)
        rule_funcs.append(rule_func)
    return name, rule_funcs

def process(part, workflow='in'):
    for rule_func in workflow_map[workflow]:
        result = rule_func(part)
        if not result:
            continue
        elif result == 'A' or result == 'R':
            return result
        else:
            return process(part, workflow=result)

# workflows, parts = test.strip().split('\n\n')

file_path = 'day_19_input.txt'
with open(file_path, 'r') as file:
    text = file.read()
    workflows, parts = text.strip().split('\n\n')


part_list = [parse_part(part) for part in parts.splitlines()]
workflow_map = dict(parse_workflow(workflow) for workflow in workflows.splitlines())

sum([sum(part.values()) for part in part_list if process(part) == 'A'])

386787

In [127]:
test = """
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}
"""

def new_part_range(part_range, key, new_range):
    part_range = part_range.copy()
    part_range[key] = new_range
    return part_range

def process_rule(part_range, rule):
    if ':' not in rule:
        return [(rule, part_range)]
        
    condition, result = rule.split(':')
    
    key = condition[0]
    comparison = condition[1]
    amount = int(condition[2:])

    start, end = part_range[key]
    workflows = []
    if (start < amount < end):
        if comparison == '<':
            workflows.append((result, new_part_range(part_range, key, (start, amount - 1))))
            workflows.append((None, new_part_range(part_range, key, (amount, end))))
        elif comparison == '>':
            workflows.append((None, new_part_range(part_range, key, (start, amount))))
            workflows.append((result, new_part_range(part_range, key, (amount + 1, end))))
    else:
        workflows.append((None, part_range))
        
    return workflows

#{workflow: part}

def parse_workflow(workflow):
    name, rules = workflow.split('{')
    rules = rules.strip('}').split(',')
    rule_funcs = []
    for rule in rules:
        rule_funcs.append(partial(process_rule, rule=rule))
    return name, rule_funcs

def process(part_range, workflow='in'):
    for rule_func in workflow_map[workflow]:
        results = rule_func(part_range)
        for result, new_part_range in results:
            if result == 'A':
                accepted.append(new_part_range)
            elif result == 'R':
                rejected.append(new_part_range)
            elif result is None:
                part_range = new_part_range
            else:
                process(new_part_range, result)

# workflows = test.strip().splitlines()

file_path = 'day_19_input.txt'
with open(file_path, 'r') as file:
    text = file.read()
    workflows, _ = text.strip().split('\n\n')


workflow_map = dict(parse_workflow(workflow) for workflow in workflows.splitlines())

accepted = []
rejected = []

part_range = {'x': (1, 4000), 'm': (1, 4000), 'a': (1, 4000), 's': (1, 4000)}
process(part_range)
len(accepted)

602

In [128]:
def combos(part_range):
    result = 1
    for start, end in part_range.values():
        diff = end - start + 1
        result *= diff
    return result

sum([combos(part_range) for part_range in accepted])

131029523269531